In [ ]:
import os
import pandas as pd
import networkx as nx
import kagglehub

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:

# Download latest version
path = kagglehub.dataset_download("khusheekapoor/coursera-courses-dataset-2021")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/khusheekapoor/coursera-courses-dataset-2021/versions/1


In [ ]:
os.listdir(path)

coursers = os.listdir(path)
for course in coursers:
    df= pd.read_csv(os.path.join(path,course))


df.head()
# import pandas as pd

# # Load the dataset
# df = pd.read_csv(path)  # Replace with the actual file name
# print(df.head())

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


## Info of the dataset using <code> info() </code>

In [ ]:
print(f' number of examples: {len(df)}')
df.info()

 number of examples: 3522
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522 entries, 0 to 3521
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Course Name         3522 non-null   object
 1   University          3522 non-null   object
 2   Difficulty Level    3522 non-null   object
 3   Course Rating       3522 non-null   object
 4   Course URL          3522 non-null   object
 5   Course Description  3522 non-null   object
 6   Skills              3522 non-null   object
dtypes: object(7)
memory usage: 192.7+ KB


## Description of the dataset and its statistics using <code> describe() </code>

In [ ]:
df.describe()

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
count,3522,3522,3522,3522,3522,3522,3522
unique,3416,184,5,31,3424,3397,3424
top,Google Cloud Platform Fundamentals: Core Infra...,Coursera Project Network,Beginner,4.7,https://www.coursera.org/learn/gcp-fundamentals,This course introduces you to important concep...,Google Cloud Platform Big Data Cloud Infrast...
freq,8,562,1444,740,8,8,8


## Missing values :)

### Printing the number of missing values

In [ ]:
nullData = df.isnull().sum()
nullData
print(nullData)

print(f"\n **************************************\n\n Number of empty records = {df.isnull().all(axis=1).sum()}")
df[df.isnull().all(axis=1)]


Course Name           0
University            0
Difficulty Level      0
Course Rating         0
Course URL            0
Course Description    0
Skills                0
dtype: int64

 **************************************

 Number of empty records = 0


,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills


## Outliers

In [ ]:
# import seaborn as sns
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

# # Create a figure and axis objects for subplots
# fig, axs = plt.subplots(1, 4, figsize=(10, 5))


# df.boxplot(column=['Difficulty level'], ax=axs[0], grid=False)
# axs[0].set_title('Age')


# df.boxplot(column=['Course Rating '], ax=axs[1], grid=False)
# axs[1].set_title('Fare')

# # df.boxplot(column=['parch'], ax=axs[2], grid=False)
# # axs[2].set_title('parch')

# # df.boxplot(column=['sibsp'], ax=axs[3], grid=False)
# # axs[3].set_title('sibsp')


# plt.tight_layout()


# plt.show()

# df.boxplot(by ='pclass', column =['fare'], grid = False)

# unique values

In [ ]:
# group_of_features= [
#     # 'Course Name','University',
#                     'Difficulty Level','Course Rating','Course URL','Course Description','Skills']

# # for column in df.columns:
# for column in group_of_features:
#     unique_values = df[column].unique()
#     print(f"\n **********\t Unique values in column '{column}': {unique_values}")

# replacing "invalid" data

In [ ]:
df['Course Rating'].replace("Not Calibrated", 0, inplace=True)
df['Course Rating'] = df['Course Rating'].astype(float)

<ipython-input-107-4bef0b91cca4>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Course Rating'].replace("Not Calibrated", 0, inplace=True)


#duplicate rows

In [ ]:
df['Course Name'].unique()

array(['Write A Feature Length Screenplay For Film Or Television',
       'Business Strategy: Business Model Canvas Analysis with Miro',
       'Silicon Thin Film Solar Cells', ...,
       'Business intelligence and data analytics: Generate insights',
       'Rigid Body Dynamics',
       'Architecting with Google Kubernetes Engine: Production'],
      dtype=object)

In [ ]:
duplicates = df[df.duplicated()]
# print(duplicates)
duplicates

# duplicates = df[df.duplicated(subset=['Course Name'], keep=False)]
# # print(duplicates)
# duplicates

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
44,Introduction to Cybersecurity Tools & Cyber At...,IBM,Beginner,4.3,https://www.coursera.org/learn/introduction-cy...,This course gives you the background needed to...,cyber-security regulation cyber security stan...
158,Guided Imagery,University of Minnesota,Intermediate,4.8,https://www.coursera.org/learn/guided-imagery,"In this course, you will learn how you can use...",functional medicine music meditation counse...
160,Teach English Now! Capstone Project 1,Arizona State University,Advanced,4.8,https://www.coursera.org/learn/teaching-englis...,"In the final Capstone Project, you will apply ...",curiosity language english language lesson ...
163,Building Resilient Streaming Analytics Systems...,Google Cloud,Advanced,4.6,https://www.coursera.org/learn/streaming-analy...,*Note: this is a new course with updated conte...,Cloud Computing bigquery bigtable dataflow ...
213,Introduction to Ableton Live,Berklee College of Music,Advanced,4.8,https://www.coursera.org/learn/ableton-live,The new version of Introduction to Ableton Liv...,music Troubleshooting Video Editing saving ...
...,...,...,...,...,...,...,...
3470,Google Cloud Platform Fundamentals: Core Infra...,Google Cloud,Conversant,4.7,https://www.coursera.org/learn/gcp-fundamentals,This course introduces you to important concep...,Google Cloud Platform Big Data Cloud Infrast...
3491,Applied Data Science Capstone,IBM,Beginner,4.6,https://www.coursera.org/learn/applied-data-sc...,This capstone project course will give you a t...,Algorithms data clustering algorithms web a...
3499,Google Cloud Platform Fundamentals: Core Infra...,Google Cloud,Conversant,4.7,https://www.coursera.org/learn/gcp-fundamentals,This course introduces you to important concep...,Google Cloud Platform Big Data Cloud Infrast...
3500,Google Cloud Platform Fundamentals: Core Infra...,Google Cloud,Conversant,4.7,https://www.coursera.org/learn/gcp-fundamentals,This course introduces you to important concep...,Google Cloud Platform Big Data Cloud Infrast...


In [ ]:
df.drop_duplicates(inplace=True)
df.describe()

,Course Rating
count,3424.000000
mean,4.443224
std,0.773170
min,0.000000
25%,4.400000
50%,4.600000
75%,4.700000
max,5.000000


In [ ]:
df

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...
...,...,...,...,...,...,...,...
3517,"Capstone: Retrieving, Processing, and Visualiz...",University of Michigan,Beginner,4.6,https://www.coursera.org/learn/python-data-vis...,"In the capstone, students will build a series ...",Databases syntax analysis web Data Visuali...
3518,Patrick Henry: Forgotten Founder,University of Virginia,Intermediate,4.9,https://www.coursera.org/learn/henry,"�Give me liberty, or give me death:� Rememberi...",retirement Causality career history of the ...
3519,Business intelligence and data analytics: Gene...,Macquarie University,Advanced,4.6,https://www.coursera.org/learn/business-intell...,�Megatrends� heavily influence today�s organis...,analytics tableau software Business Intellig...
3520,Rigid Body Dynamics,Korea Advanced Institute of Science and Techno...,Beginner,4.6,https://www.coursera.org/learn/rigid-body-dyna...,"This course teaches dynamics, one of the basic...",Angular Mechanical Design fluid mechanics F...


# adding an id to the courses

In [ ]:
import uuid
df['ID'] = range(1, len(df) + 1)

df

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills,ID
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...,1
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...,2
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...,3
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...,4
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...,5
...,...,...,...,...,...,...,...,...
3517,"Capstone: Retrieving, Processing, and Visualiz...",University of Michigan,Beginner,4.6,https://www.coursera.org/learn/python-data-vis...,"In the capstone, students will build a series ...",Databases syntax analysis web Data Visuali...,3420
3518,Patrick Henry: Forgotten Founder,University of Virginia,Intermediate,4.9,https://www.coursera.org/learn/henry,"�Give me liberty, or give me death:� Rememberi...",retirement Causality career history of the ...,3421
3519,Business intelligence and data analytics: Gene...,Macquarie University,Advanced,4.6,https://www.coursera.org/learn/business-intell...,�Megatrends� heavily influence today�s organis...,analytics tableau software Business Intellig...,3422
3520,Rigid Body Dynamics,Korea Advanced Institute of Science and Techno...,Beginner,4.6,https://www.coursera.org/learn/rigid-body-dyna...,"This course teaches dynamics, one of the basic...",Angular Mechanical Design fluid mechanics F...,3423


In [ ]:
df['Combined Features'] = df['Course Description']

# Step 2: Convert the combined text into TF-IDF features
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Combined Features'])


In [ ]:
df['Combined Features'].head()

,Combined Features
0,Write A Feature Length Screenplay For Film Or ...
1,Business Strategy: Business Model Canvas Analy...
2,Silicon Thin Film Solar Cells This course cons...
3,"Finance for Managers When it comes to numbers,..."
4,Retrieve Data using Single-Table SQL Queries I...


In [ ]:
similarity_matrix = cosine_similarity(tfidf_matrix)

In [ ]:
df[df['ID'] == 2].index[0]

1

In [ ]:
def recommended_courses(course_id, similarity_matrix, df, top_n=5):
      course_idx = df[df['ID'] == course_id].index[0]
      similarity_scores = list(enumerate(similarity_matrix[course_idx]))
      sorted_courses = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:3+1]
      recommended_course_ids = [df.iloc[i[0]]['ID'] for i in sorted_courses]
      return recommended_course_ids, sorted_courses

In [ ]:
course_id = 20
print(df.loc[course_id - 1, 'Course URL'])
df.loc[course_id - 1]

https://www.coursera.org/learn/recommendation-models-gcp


,19
Course Name,Recommendation Systems with TensorFlow on GCP
University,Google Cloud
Difficulty Level,Advanced
Course Rating,4.2
Course URL,https://www.coursera.org/learn/recommendation-...
Course Description,"In this course, you'll apply your knowledge of..."
Skills,systems architecture Cloud Computing Google ...
ID,20
Combined Features,Recommendation Systems with TensorFlow on GCP ...


In [ ]:
ids, similarity = recommended_courses(course_id, similarity_matrix, df)
for id in ids:
  print(df.loc[id, 'Course Name'])
  print(df.loc[id, 'Course URL'])

print(similarity)

FinTech and the Transformation in Financial Services
https://www.coursera.org/learn/fintech-transformation-financial-services
Strengthening Your Widening Network
https://www.coursera.org/learn/strengthening-your-widening-network
Python Geospatial Data Analysis
https://www.coursera.org/learn/python-geospatial-data-analysis
[(1497, 0.4153458785073375), (2112, 0.3970414023741761), (891, 0.33709519884479633)]


In [ ]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('wordnet2022')

! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet # temp fix for lookup error.

cp: cannot stat '/usr/share/nltk_data/corpora/wordnet2022': No such file or directory


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to /root/nltk_data...
[nltk_data]   Package wordnet2022 is already up-to-date!


In [ ]:
import pickle
pickle.dump(similarity_matrix, open('similarity_matrix.pkl', 'wb'))

In [ ]:
# Save to a CSV file
df.to_csv("processed_data.csv", index=False)
